# Solving SAT With Grover
In this notebook, we aim to solve satisfyability formulae using grover search.
For this, we first construct marking oracles for the basic gates of AND, OR and NOT.
Then, we transform them into the required phase-oracle-form.

The main resources used are this [Microsoft Kata](https://github.com/microsoft/QuantumKatas/blob/ec925ccfcb599a6bb29c8a39035d0b149f447f9a/SolveSATWithGrover/Workbook_SolveSATWithGrover.ipynb) and this [Qiskit Textbook Chapter](https://qiskit.org/textbook/ch-applications/satisfiability-grover.html).



In [ ]:
from IPython.core import page
page.page = print

from qiskit.circuit import Qubit, QuantumRegister, AncillaRegister, QuantumCircuit, Gate, ClassicalRegister
from qiskit.quantum_info import Operator

import numpy as np
import math
np.set_printoptions(threshold=1e6)

## Marking Oracles
### And

In [ ]:
from grover_sat import create_and_oracle
%psource create_and_oracle

### Or

In [ ]:
from grover_sat import create_or_oracle
%psource create_or_oracle

While we could go on further and create _XOR_ and _AlternatingBits_ oracles but we don't need them for now so let's not :)

## SAT Marking Oracles
Using our marking gate-oracles, we can create oracles for evaluating SAT clauses and finally complete functions.

### Single Clause Oracle
A Clause is a disjunction of variables (qubits) that are potentially negated.
The clause `x0 || !x1` may be represented by the input `[(0, true), (1, false)]`.

In [ ]:
from grover_sat import get_clause_qubits
%psource create_and_oracle

from grover_sat import create_clause_oracle
%psource create_and_oracle

In [ ]:
# Debug Clause Oracle
inp_reg = QuantumRegister(3, name="q_in")
tar = Qubit()
tar_reg = QuantumRegister(bits=[tar])
clause = [(0, True),(1, False),(2,False)]

clause_circuit = create_clause_oracle(inp_reg, tar, clause)
clause_circuit.draw(output="mpl")

### k-SAT Oracle
Using our single clause oracle, we can take their conjunction and create a marking oracle for the entire expression.

In [ ]:
from grover_sat import create_ksat_oracle
%psource create_ksat_oracle

**DONE** We can now formulate a SAT problem and create a marking oracle for it!

In [ ]:
# f(x) = (B + !D + E) * (A + !C)
# g(x) = (!A+!C)(!B+C)
problem_f = [[(1, True),(3, False),(4, True)],
          [(0, True),(2, False)]]

problem_g = [[(0, False)], [(1, True), (2, True)]]

problem = np.array(problem_f, dtype=object)
n_variables = 5

inp_reg = QuantumRegister(n_variables, name="q_in")
tar = Qubit()
tar_reg = QuantumRegister(bits=[tar])

problem_circuit = create_ksat_oracle(inp_reg, tar, problem)
problem_circuit.draw(output="mpl", justify="left")

## Converting the oracles into phase oracles for Grover

In [ ]:
from grover_sat import oracle_converter
%psource oracle_converter

In [ ]:
phase_problem_circuit = oracle_converter(problem_circuit, len(inp_reg))
phase_problem_circuit.draw(output="mpl")

### Analyzing Oracles

In [ ]:
from grover_sat import print_matrix
%psource print_matrix
    
# print_matrix(phase_problem_circuit)

In [ ]:
from grover_sat import print_diagonal_analysis
%psource print_diagonal_analysis
        
#count_dummy = {'101': 30, '111': 32, '011': 31, '100': 271, '110': 296, '001': 28, '010': 282, '000': 30}
count_dummy = {'01010': 1, '11000': 2, '11011': 7, '00011': 1, '01111': 3, '00010': 8, '00101': 4, '10101': 5, '01011': 6, '00001': 8, '11010': 5, '10010': 10, '00000': 1, '01001': 87, '10000': 6, '11001': 4, '11111': 2, '10110': 89, '11101': 3, '00110': 75, '11110': 87, '10111': 5, '01101': 79, '01110': 87, '10011': 7, '00100': 78, '01100': 78, '10001': 3, '01000': 91, '11100': 72, '00111': 4, '10100': 82}
# print_diagonal_analysis(phase_problem_circuit, count_dummy)

# Plugging Oracles into Grover
As we can now create phase oracles, we can model a complete grover loop.

1. Create uniform superposition over q_in (an & target stay |0>)
2. Make oracle: ksat_for_problem(...) & phase_oracle(...)
3. Grover Loop (k-times? correctness-check?)
    1. Append Phase_Oracle over all q registers
    2. Append Grover Diffuser over all q registers
3. Add classical registers for output measurements
5. Measure q_in into classical registers and check solution (histogram)

In [ ]:
# Input:
# f(x) = (x0 + !x1) * (!x2 + !x3) * (x2 + x3) * (!x4 + x5) # Poster Problem
#problem = [[(0, True), (1, True)], [(2, False), (3, False)], [(2, True), (3, True)], [(4, False), (5, True)]]
# n_variables = 6

(1.) Create uniform superposition

In [ ]:
from fragments.quantum_states import add_all_hadamards
%psource add_all_hadamards

(2.) Make oracle for specific sat problem

In [ ]:
from grover_sat import init_sat_circuit
%psource init_sat_circuit

In [ ]:
_, _, _, _, qc_phase_oracle = init_sat_circuit(problem)
qc_phase_oracle_decomposed = qc_phase_oracle.decompose() # .decompose().decompose().decompose()
qc_phase_oracle_decomposed.draw(output="mpl", justify="left")


(3.) Grover Loop

In [ ]:
from grover_sat import create_ksat_grover
%psource create_ksat_grover

In [ ]:
# A suitable k could be calculated using quantum phase estimation
k=1

# Create the circuit
main_qc, qc_oracle = create_ksat_grover(problem, k)


main_qc.draw(output="mpl", justify="left")

The Quantum Circuit is completed! Simulate it!

In [ ]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import StatevectorSimulator

transpiled_grover_circuit = transpile(main_qc, StatevectorSimulator())
results = StatevectorSimulator().run(transpiled_grover_circuit, shots=1000).result()
counts = results.get_counts()
histogram = plot_histogram(counts, figsize=(40, 10))
histogram # For display purposes in jupyter

## Warmstarting other Quantum circuits
If we want to use this result to warmstart other quantum circuits, we need the list of complex amplitudes of that statevector!

In [ ]:
# Get the statevector from counts.
from grover_sat import calc_statevector_from
%psource calc_statevector_from

This function can be used to calculate a statevector from a set of measurements!
Now double check that this vector yields the same result

In [ ]:
num_vars = n_variables # restrict to n qubits defined near the problems because we don't care about our ancillas
manual_statevector = calc_statevector_from(counts, num_vars)
print(repr(manual_statevector))

qc_vec = QuantumCircuit(QuantumRegister(num_vars), ClassicalRegister(num_vars))
qc_vec.initialize(manual_statevector)
qc_vec.measure(list(range(num_vars)), list(range(num_vars)))
qc_vec.draw(output="mpl")

In [ ]:
transpiled_vec_circuit = transpile(qc_vec, StatevectorSimulator())
print(f"Circuit Depth: {transpiled_vec_circuit.depth()}")
results = StatevectorSimulator().run(transpiled_vec_circuit, shots=1000).result()
counts = results.get_counts()
histogram = plot_histogram(counts, figsize=(40, 10))
histogram # For display purposes in jupyter

# Importing Feature Models

In [ ]:
# from qiskit import Aer, transpile
# from util.xml_reader import Extended_Modelreader
# from util.dimacs_reader import DimacsReader
# from util.cnf import CNF
# import os 

from grover_sat import create_grover_for_model
%psource create_grover_for_model
    
model = "../benchmarks/featureide-examples/car.dimacs"
# model = "../benchmarks/problem_f.cnf"
model_circuit = create_grover_for_model(model)
# model_circuit.draw(output="mpl")

In [ ]:
from grover_sat import collect_circuit_info
%psource collect_circuit_info

# gather backend metrics
# from qiskit import IBMQ, assemble, transpile
# provider = IBMQ.load_account()

# print(provider.backends())
# simulate_circuit(model_circuit, provider.backend.ibmq_geneva)    

# actual simulation
# info = collect_circuit_info(model_circuit, backend="statevector_simulator", shots=1000, simulate=True)
# plot_histogram(info['counts'], figsize=(40, 10))